In [1]:
%matplotlib inline

import os
import sys
genre_class_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Loading Genre_Classification system from path", genre_class_path)
sys.path.append(genre_class_path)

import numpy as np
import pandas as pd
import tensorflow as tf
import json

Loading AI Lit system from path /mnt/d/UNIV NOTES/Y4S1/DSA4199/ai-lit


2022-10-25 00:35:57.291725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-25 00:35:57.293851: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kathyij74/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
dataset_wkspc = os.path.join(genre_class_path, 'workspace', 'gb_input')
train_json_path = os.path.join(dataset_wkspc, 'train_index_list.json')

with open(train_json_path) as f:
    train_json = json.load(f)

In [4]:
train_json[0]['body']

'project gutenberg\'s the house on the borderland, by william hope hodgson\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: the house on the borderland\r\n\r\nauthor: william hope hodgson\r\n\r\nrelease date: november 10, 2003 [ebook #10002]\r\nlast updated: january 19, 2009\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: ascii\r\n\r\n*** start of this project gutenberg ebook the house on the borderland ***\r\n\r\n\r\n\r\n\r\nproduced by suzanne shell, sjaani and pg distributed proofreaders\r\n\r\n\r\n\r\n\r\nthe house on the borderland\r\n\r\nwilliam hope hodgson\r\n\r\n\r\n_from the manuscript discovered in 1877 by messrs. tonnison and\r\nberreggnog in the ruins that lie to the south of the village of\r\nkraighten, in the west of ireland. set out

In [5]:
train_json[0]

{'index': '10002',
 'indexfile': 'Gutenberg/cache/epub/10002/pg10002.rdf',
 'title': 'the house on the borderland',
 'author': 'hodgson, william hope',
 'subjects': ['Science fiction'],
 'bodyfile': 'Gutenberg/aleph.gutenberg.org/1/0/0/0/10002/10002.zip',
 'body': 'project gutenberg\'s the house on the borderland, by william hope hodgson\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: the house on the borderland\r\n\r\nauthor: william hope hodgson\r\n\r\nrelease date: november 10, 2003 [ebook #10002]\r\nlast updated: january 19, 2009\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: ascii\r\n\r\n*** start of this project gutenberg ebook the house on the borderland ***\r\n\r\n\r\n\r\n\r\nproduced by suzanne shell, sjaani and pg distributed proofreade

In [6]:
data_try = train_json[0]['body']

# Step - a : Remove blank rows if any.
# data_try.dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
data_try = data_try.lower()
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
data_try= word_tokenize(data_try)
# Step - d : POS Tagging
data_try = pos_tag(data_try)

In [7]:
print(data_try)

[('project', 'NN'), ('gutenberg', 'NN'), ("'s", 'POS'), ('the', 'DT'), ('house', 'NN'), ('on', 'IN'), ('the', 'DT'), ('borderland', 'NN'), (',', ','), ('by', 'IN'), ('william', 'JJ'), ('hope', 'NN'), ('hodgson', 'NN'), ('this', 'DT'), ('ebook', 'NN'), ('is', 'VBZ'), ('for', 'IN'), ('the', 'DT'), ('use', 'NN'), ('of', 'IN'), ('anyone', 'NN'), ('anywhere', 'RB'), ('at', 'IN'), ('no', 'DT'), ('cost', 'NN'), ('and', 'CC'), ('with', 'IN'), ('almost', 'RB'), ('no', 'DT'), ('restrictions', 'NNS'), ('whatsoever', 'RB'), ('.', '.'), ('you', 'PRP'), ('may', 'MD'), ('copy', 'VB'), ('it', 'PRP'), (',', ','), ('give', 'VB'), ('it', 'PRP'), ('away', 'RB'), ('or', 'CC'), ('re-use', 'VB'), ('it', 'PRP'), ('under', 'IN'), ('the', 'DT'), ('terms', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('project', 'NN'), ('gutenberg', 'NN'), ('license', 'NN'), ('included', 'VBD'), ('with', 'IN'), ('this', 'DT'), ('ebook', 'NN'), ('or', 'CC'), ('online', 'NN'), ('at', 'IN'), ('www.gutenberg.org', 'JJ'), ('title', 'NN'), ('

In [8]:
data_tags = [t for t in data_try if ('V' in t[1])]

In [9]:
print(data_tags)

[('is', 'VBZ'), ('copy', 'VB'), ('give', 'VB'), ('re-use', 'VB'), ('included', 'VBD'), ('hope', 'VBP'), ('ebook', 'VBD'), ('set', 'VBN'), ('gutenberg', 'VBZ'), ('ebook', 'VB'), ('produced', 'VBN'), ('distributed', 'VBD'), ('william', 'VBZ'), ('hope', 'VBP'), ('discovered', 'VBN'), ('lie', 'VBP'), ('set', 'VBN'), ('tread', 'VBP'), ('_', 'VBP'), ('listen', 'VB'), ('vanishing', 'VBG'), ('shoon', 'VB'), ('tread', 'VBP'), ('die', 'VB'), ('are', 'VBP'), ('have', 'VBP'), ('pondered', 'VBN'), ('is', 'VBZ'), ('set', 'VBN'), ('i', 'VB'), ('are', 'VBP'), ('prompt', 'VBP'), ('leave', 'VB'), ('was', 'VBD'), ('handed', 'VBN'), ('picture', 'VB'), ('was', 'VBD'), ('given', 'VBN'), ('turning', 'VBG'), ('making', 'VBG'), ('is', 'VBZ'), ('thick', 'VB'), ('save', 'VBP'), ('filled', 'VBN'), ('writ', 'VB'), ('have', 'VBP'), ('write', 'VBP'), ('have', 'VBP'), ('read', 'VBP'), ('lifted', 'VBD'), ('blind', 'VBP'), ('looked', 'VBD'), ('i', 'VBP'), ('wandered', 'VBN'), ('had', 'VBD'), ('charge', 'VB'), ('is', 'V

In [12]:
verb = []
for data in data_tags:
    verb.append(data[0])
verb = [*set(verb)]
print(verb)

['agree', 'steady', 'asking', 'ached', 'increasing', 'fade', 'escaping', 'went', 'sniffing', 'times', 'maintained', 'gasping', 'fascinated', 'growing', 'caused', 'riven', 'darted', 'derive', 'existing', 'understands', 'shed', 'heard', 'commences', 'work', 'prosecute', 'perceived', 'smashing', 'force', 'performed', 'dwindle', 'covered', 'brace', 'picking', 'copy', 'remove', 'download', 'hailed', 'imagined', '....', 'thrust', 'ashlike', 'crumbled', 'wooded', 'skipping', 'annihilated', 'camp', 'wandered', 'dressing', 'tumbled', 'speaking', 'side', 'remote', 'hunt', 'increase', 'pressed', 'saddened', 'taken', 'slipped', 'writ', 'whispered', 'require', 'undulating', 'deadening', 'neared', 'scintillating', 'rush', 'dropping', 'blustering', '_vi_', 'governing', 'wedged', 'cellar', 'approaching', 'chasing', 'ventured', 'muddled', 'caught', 'happening', 'traversing', 'deny', 'tracing', 'drown', 'fail', 'distributing', 'owed', 'prevailed', 'remained', 'is', 'dawning', 'accomplish', 'ascended', '

In [13]:
# create a function to filter the verbs in every body into a list
def verb_select(text):
    text = text.lower()
    text = word_tokenize(text)
    text = pos_tag(text)
    data_tags = [t for t in text if ('V' in t[1])]
    verb = []
    for data in data_tags:
        verb.append(data[0])
    verb = [*set(verb)]
    return verb

In [23]:
verb_select(train_json[0]['body'])

['agree',
 'steady',
 'asking',
 'ached',
 'increasing',
 'fade',
 'escaping',
 'went',
 'sniffing',
 'times',
 'maintained',
 'gasping',
 'fascinated',
 'growing',
 'caused',
 'riven',
 'darted',
 'derive',
 'existing',
 'understands',
 'shed',
 'heard',
 'commences',
 'work',
 'prosecute',
 'perceived',
 'smashing',
 'force',
 'performed',
 'dwindle',
 'covered',
 'brace',
 'picking',
 'copy',
 'remove',
 'download',
 'hailed',
 'imagined',
 '....',
 'thrust',
 'ashlike',
 'crumbled',
 'wooded',
 'skipping',
 'annihilated',
 'camp',
 'wandered',
 'dressing',
 'tumbled',
 'speaking',
 'side',
 'remote',
 'hunt',
 'increase',
 'pressed',
 'saddened',
 'taken',
 'slipped',
 'writ',
 'whispered',
 'require',
 'undulating',
 'deadening',
 'neared',
 'scintillating',
 'rush',
 'dropping',
 'blustering',
 '_vi_',
 'governing',
 'wedged',
 'cellar',
 'approaching',
 'chasing',
 'ventured',
 'muddled',
 'caught',
 'happening',
 'traversing',
 'deny',
 'tracing',
 'drown',
 'fail',
 'distribut

In [24]:
train_with_verbs = {}
for entry in train_json:
    verb_list = [verb_select(entry['body'])]
    train_with_verbs.update({entry['index']: verb_list})

In [25]:
with open('verb_list.json', 'w') as f:
    json.dump(train_with_verbs, f)

In [17]:
train_with_verbs['10005']

['profaning',
 'executes',
 'adapted',
 'encouraging',
 'agree',
 'bubbling',
 'spendthrift._',
 'asking',
 'rescue',
 'deformed',
 'landing',
 'increasing',
 'manifesting',
 'escaping',
 'went',
 'lavished',
 'times',
 'reprinted',
 'contracting',
 'commands',
 'exercised',
 'maintained',
 'contending',
 "'for",
 'partake',
 'divine',
 'caused',
 'derive',
 'existing',
 'neglected',
 'heard',
 'work',
 'direct',
 'rent',
 'unscrewed',
 'transfers',
 'perceived',
 'performed',
 'force',
 'prize',
 'endanger',
 'covered',
 'picking',
 'copy',
 'remove',
 'divided',
 'download',
 'vex',
 'prepares',
 'strained',
 'resisted',
 'disqualified',
 'imagined',
 'justified',
 '_need_',
 'convince',
 'ingratiating',
 'afford',
 'forbidden',
 'atte',
 'split',
 'adopted',
 'liberated',
 'rubbing',
 'published',
 'enlarging',
 'wandered',
 'recognising',
 'digging',
 'spared',
 'waking',
 'glad',
 'speaking',
 'grind',
 'communicated',
 'remote',
 'reproach',
 'insults',
 'carries',
 'increase',
 

In [27]:
test_json_path = os.path.join(dataset_wkspc, 'test_index_list.json')

with open(test_json_path) as f:
    test_json = json.load(f)

In [33]:
test_with_verbs = {}
for entry in test_json:
    verb_list = [verb_select(entry['body'])]
    test_with_verbs.update({entry['index']: verb_list})

In [34]:
test_with_verbs['10029']

[['encouraging',
  'agree',
  'asking',
  'corroborated',
  'divides',
  'dick',
  'increasing',
  'escaping',
  'went',
  'exercised',
  'gasping',
  'misinterpreted',
  'divine',
  'fascinated',
  'growing',
  'caused',
  'recalling',
  'derive',
  'disliking',
  'heard',
  'work',
  'unscrewed',
  'performed',
  'force',
  'copy',
  'remove',
  'identifying',
  'divided',
  'download',
  'resisted',
  'persecute',
  'imagined',
  'justified',
  'convince',
  'afford',
  'alfred',
  'elope',
  'unconvincing',
  'clapped',
  'fool',
  'sake',
  'sneering',
  'speaking',
  'glad',
  'communicated',
  'hunt',
  'pressed',
  'taken',
  'slipped',
  'surged',
  'require',
  'burk',
  'neared',
  'rush',
  'approaching',
  'interrupted',
  'ventured',
  'estimating',
  'xvi',
  'xxii',
  'muddled',
  'caught',
  'neutralized',
  'constitute',
  'happening',
  'boxing',
  'weighted',
  'deny',
  'settle',
  'tracing',
  'fail',
  'distributing',
  'suffered',
  'congratulate',
  'indulge',


In [35]:
data_json_path = os.path.join(dataset_wkspc, 'data_index.json')

with open(data_json_path) as f:
    full_json = json.load(f)

In [37]:
full_verb_dict = {**train_with_verbs, **test_with_verbs}
full_verb_list = []

In [38]:
for entry in full_json:
    full_verb_list.append({'index': entry['index'], 'title': entry['title'], 
                            'author': entry['author'], 'subjects': entry['subjects'], 'verbs': full_verb_dict[entry['index']][0]})

In [40]:
full_verb_list[10]

{'index': '10324',
 'title': 'bull hunter',
 'author': 'brand, max',
 'subjects': ['Western stories'],
 'verbs': ['master',
  'agree',
  'asking',
  'ached',
  'splinted',
  'step',
  'increasing',
  'slave',
  'landing',
  'polish',
  'went',
  'fascinated',
  'twisting',
  'tore',
  'growing',
  'caused',
  'thoroughbred',
  'darted',
  'derive',
  'understands',
  'shed',
  'heard',
  'work',
  'smashing',
  'force',
  'performed',
  'covered',
  'picking',
  'copy',
  'dispensed',
  'remove',
  'divided',
  'download',
  'swerving',
  'figure',
  'revolting',
  'anchored',
  "worse'n",
  'convince',
  'thrust',
  'crumbled',
  'elope',
  'benumbed',
  'adopted',
  'rubbing',
  'fool',
  'riley',
  'lem',
  'wandered',
  'digging',
  'tumbled',
  'spared',
  'waking',
  'speaking',
  'glad',
  'side',
  'batter',
  'communicated',
  'loosening',
  'drunk',
  'hunt',
  'carries',
  'dan',
  'featured',
  'smoking',
  'pressed',
  'aspired',
  'slipped',
  'taken',
  'surged',
  'whis

In [41]:
with open('full_verb_list.json', 'w') as f:
    json.dump(full_verb_list, f)